## Clean your deployed model endpoint content
-----

#### In this notebook, we will parse through the existing endpoint.json to delete all of the existing endpoints once you are done with running your respective benchmarking tests.

***If you are with running all of the tests, and want to delete the existing endpoints, run this notebook.***

In [1]:
# if interactive mode is set to no -> pickup fmbench from Python installation path
# if interactive mode is set to yes -> pickup fmbench from the current path (one level above this notebook)
# if interactive mode is not defined -> pickup fmbench from the current path (one level above this notebook)
# the premise is that if run non-interactively then it can only be run through main.py which will set interactive mode to no
import os
import sys
if os.environ.get("INTERACTIVE_MODE_SET", "yes") == "yes":
    sys.path.append(os.path.dirname(os.getcwd()))

In [2]:
## Import all necessary libraries
import json
import boto3
import logging
from fmbench.utils import *
from fmbench.globals import *

config file current -> configs/config-distilbert-base-uncased.yml, None
Loaded config: {'general': {'name': 'distilbert-base-uncased-v1', 'model_name': 'distilbert-base-uncased'}, 'aws': {'region': 'us-east-1', 'sagemaker_execution_role': 'arn:aws:sts::015469603702:assumed-role/fmbench-role/SageMaker', 'bucket': 'sagemaker-fmbench-write-015469603702'}, 'dir_paths': {'data_prefix': 'data', 'prompts_prefix': 'prompts', 'all_prompts_file': 'all_prompts.csv', 'metrics_dir': 'metrics', 'models_dir': 'models', 'metadata_dir': 'metadata'}, 's3_read_data': {'read_bucket': 'sagemaker-fmbench-read-015469603702', 'scripts_prefix': 'scripts', 'script_files': ['hf_token.txt'], 'source_data_prefix': 'source_data1', 'tokenizer_prefix': 'tokenizer', 'prompt_template_dir': 'prompt_template1', 'prompt_template_file': 'prompt_template.txt'}, 'run_steps': {'1_generate_data.ipynb': True, '2_deploy_model.ipynb': True, '3_run_inference.ipynb': True, '4_model_metric_analysis.ipynb': True, '5_cleanup.ipynb': T

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


CustomTokenizer, based on HF transformers


In [3]:
## Set your logger to display all of the endpoints being cleaned
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

## Load the config file to extract our endpoint.json file and its respective file path
config = load_config(CONFIG_FILE)

In [4]:
## Refer to the file path for the endpoint
endpoint_info_list = json.loads(get_s3_object(config['aws']['bucket'], ENDPOINT_LIST_PATH))
logger.info(f"found information for {len(endpoint_info_list)} endpoints, bucket={config['aws']['bucket']}, key={ENDPOINT_LIST_PATH}")
logger.info(json.dumps(endpoint_info_list, indent=2))

[2024-03-19 20:03:12,369] p32183 {3609575848.py:3} INFO - found information for 1 endpoints, bucket=sagemaker-fmbench-write-015469603702, key=distilbert-base-uncased-v1-SageMaker/data/models/endpoints.json
[2024-03-19 20:03:12,370] p32183 {3609575848.py:4} INFO - [
  {
    "experiment_name": "distilbert-base-uncased-ml-p3-2xlarge",
    "endpoint": {
      "EndpointName": "distilbert-base-uncased-1710877330",
      "EndpointArn": "arn:aws:sagemaker:us-east-1:015469603702:endpoint/distilbert-base-uncased-1710877330",
      "EndpointConfigName": "distilbert-base-uncased-1710877330",
      "ProductionVariants": [
        {
          "VariantName": "AllTraffic",
          "DeployedImages": [
            {
              "SpecifiedImage": "763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-inference:1.10.2-transformers4.17.0-gpu-py38-cu113-ubuntu20.04",
              "ResolvedImage": "763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-inference@sha256:17e776fd3295c

In [5]:
## initialize a sagemaker client
sm_client = boto3.client("sagemaker")
# Iterate over the endpoint_info_list and mark the items for deletion
for item in endpoint_info_list:   
    
    ## Extract the endpoint name from the deployed model configuration
    ep_name = item['endpoint']["EndpointName"]
    try:
        ## Describe the model endpoint 
        logger.info(f"Going to describing the endpoint -> {ep_name}")
        resp = sm_client.describe_endpoint(EndpointName=ep_name)
        
        ## If the given model endpoint is in service, delete it 
        if resp['EndpointStatus'] == 'InService':
            logger.info(f"going to delete {ep_name}")
            ## deleting the model endpoint
            sm_client.delete_endpoint(EndpointName=ep_name)
            logger.info(f"deleted {ep_name}")
    except Exception as e:
        logger.error(f"error deleting endpoint={ep_name}, exception={e}")

[2024-03-19 20:03:12,504] p32183 {707195403.py:10} INFO - Going to describing the endpoint -> distilbert-base-uncased-1710877330
[2024-03-19 20:03:12,665] p32183 {707195403.py:15} INFO - going to delete distilbert-base-uncased-1710877330
[2024-03-19 20:03:12,770] p32183 {707195403.py:18} INFO - deleted distilbert-base-uncased-1710877330
